In [ ]:
# J. Ryu, Electron Microscopy and Spectroscopy Lab., Seoul National University
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hys
import tkinter.filedialog as tkf

In [ ]:
%matplotlib inline

In [ ]:
num_split = 3
adr = []
num_adr = []
for i in range(num_split):
    temp = tkf.askopenfilenames()
    num_adr.append(len(temp))
    adr.append(temp)
print(num_adr)
num_img = np.max(num_adr)

In [ ]:
mrad_per_pixel = 0.1 # mrad
print(mrad_per_pixel)
rel_lmb = 0.00197 # nm, relativistic wavelength for 300 kV
k_per_pixel = np.tan(mrad_per_pixel/1000) / rel_lmb# 1/nm
print(k_per_pixel)
r_per_pixel = 0.400 # nm

s_dim = 80
cb_rad = 20
k_list = np.arange(s_dim)*k_per_pixel
print(k_list)
print(k_list.shape)
r_list = np.arange(s_dim)
print(r_list)
print(r_list.shape)

In [ ]:
def rif_to_rdf(rif, r_list, k_list):
    gr = []
    profile = rif / np.max(rif)
    for i in range(len(r_list)):
        sin_rk = np.sin(r_list[i]*k_list)
        rif_sin = np.multiply(profile, sin_rk)
        gr_tmp = 8 * np.pi * np.trapz(rif_sin, dx=(k_list[1]-k_list[0]))
        gr.append(gr_tmp)
    
    return np.asarray(gr)

In [ ]:
data_by_split = []
gr_by_split = []
data_shape = []
for i in range(num_split):
    temp = []
    temp_gr = []
    temp_shape = []
    for path in adr[i]:
        print(path)
        tmp = hys.load(path).data
        print(tmp.shape)
        if len(np.where(tmp[0, 0] == 0.0)[0]) != cb_rad:
            tmp = np.insert(tmp, 0, 0, axis=2)        
        
        tmp = tmp[:, :, :s_dim]
        print(tmp.shape)
        
        gr_tmp = []
        for j in range(tmp.shape[0]):
            for k in range(tmp.shape[1]):
                gr_tmp.append(rif_to_rdf(tmp[j ,k], r_list, k_list))
        gr_tmp = np.asarray(gr_tmp).reshape(tmp.shape[0], tmp.shape[1], -1)   
        
        temp.append(tmp)
        temp_gr.append(gr_tmp)
        temp_shape.append(tmp.shape)
    data_by_split.append(temp)
    gr_by_split.append(temp_gr)
    data_shape.append(temp_shape)

In [ ]:
data_shape = np.asarray(data_shape)
print(data_shape.shape)
#print(np.unique(data_shape[:, :, 2]))
#s_dim = np.min(data_shape[:, :, 2])
print(s_dim)

In [ ]:
color_rep = ["green", "red", "blue", "purple", "orange", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "black", "darkgray"]
print(len(color_rep))

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib qt

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    for img in data_by_split[i]:
        tmp = img / np.max(img, axis=2)[:, :, np.newaxis]
        line = np.mean(tmp, axis=(0, 1))
        #line = line / np.max(line)
        ax.plot(line, color=color_rep[i], linestyle="-", label="split %d"%(i+1))
ax.grid()
fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    temp = []
    for arr in data_by_split[i]:
        temp.extend(arr.reshape(-1, s_dim))
    temp = np.asarray(temp)
    temp = temp / np.max(temp, axis=1)[:, np.newaxis]
    line = np.mean(temp, axis=0)
    #line = line / np.max(line)
    ax.plot(line, color=color_rep[i], linestyle="-", label="split %d"%(i+1))
ax.grid()
fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    for img in gr_by_split[i]:
        tmp = img / np.max(img, axis=2)[:, :, np.newaxis]
        line = np.mean(tmp, axis=(0, 1))
        #line = line / np.max(line)
        ax.plot(line, color=color_rep[i], linestyle="-", label="split %d"%(i+1))
ax.grid()
fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.tick_params(axis="x", labelsize=15)
for i in range(num_split):
    temp = []
    for arr in gr_by_split[i]:
        temp.extend(arr.reshape(-1, s_dim))
    temp = np.asarray(temp)
    temp = temp / np.max(temp, axis=1)[:, np.newaxis]
    line = np.mean(temp, axis=0)
    #line = line / np.max(line)
    ax.plot(line, color=color_rep[i], linestyle="-", label="split %d"%(i+1))
ax.grid()
fig.legend(fontsize="xx-large")
fig.tight_layout()
plt.show()